In [3]:
import torch
from diffusers import DiffusionPipeline, LCMScheduler,StableDiffusionPipeline,DDIMScheduler,StableDiffusionXLPipeline,AutoPipelineForText2Image
from diffusers.utils import make_image_grid
import numpy as np
from AK4Prompts import AK4Prompts
from AK4Prompts_pipeline import AK4PromptsPipeline

device = 'cuda:0'

pipeline= AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16)
# SD1.5+LCM-LoRA 
# pipeline = AutoPipelineForText2Image.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
# pipeline.load_lora_weights("latent-consistency/lcm-lora-sdv1-5")
# pipeline.scheduler = LCMScheduler.from_config(pipeline.scheduler.config)
pipeline = pipeline.to(device)

ak4prompts = AK4Prompts().to(device)
ak4prompts_path = "/checkpoints/SD1.5_LCMLoRA_S4_aes1_clip2.25_hps2.25/pytorch_model.bin"
ak4prompts.load_state_dict(torch.load(ak4prompts_path))
ak4prompts_pipeline = AK4PromptsPipeline(pipeline=pipeline,ak4prompts=ak4prompts,keywords_filename="keywords_list.txt")

prompt = "vase of mixed flowers"

scores_weights = {'aesthetic':1,'clip':5,'hps':3}
prompt_with_keywords = ak4prompts_pipeline.keywords_ranking(prompt=prompt,scores_weights=scores_weights,topk=10)

image = pipeline(
    prompt=prompt_with_keywords, num_inference_steps=4, guidance_scale=1.0
    ).images[0]
image.show()


Loading pipeline components...: 100%|██████████| 7/7 [00:02<00:00,  3.37it/s]


In [ ]:
# the evolution of image generated by setting different degrees of specific preferences
prompts = "vase of mixed flowers"

generator = torch.manual_seed(1024)

hps_scores_weights= [{'aesthetic':0,'clip':1,'hps':-100},
                     {'aesthetic':0,'clip':1,'hps':-3},
                     {'aesthetic':0,'clip':1,'hps':-1},
                     {'aesthetic':0,'clip':1,'hps':-0.5},
                     {'aesthetic':0,'clip':1,'hps':0},
                     {'aesthetic':0,'clip':1,'hps':0.5},
                     {'aesthetic':0,'clip':1,'hps':1},
                     {'aesthetic':0,'clip':1,'hps':3},
                     {'aesthetic':0,'clip':1,'hps':100}]

# aes_scores_weights= [{'aesthetic':-100,'clip':1,'hps':0},
#                      {'aesthetic':-3,'clip':1,'hps':0},
#                      {'aesthetic':-1,'clip':1,'hps':0},
#                      {'aesthetic':-0.5,'clip':1,'hps':0},
#                      {'aesthetic':0,'clip':1,'hps':0},
#                      {'aesthetic':0.5,'clip':1,'hps':0},
#                      {'aesthetic':1,'clip':1,'hps':0},
#                      {'aesthetic':3,'clip':1,'hps':0},
#                      {'aesthetic':100,'clip':1,'hps':0}]

# clip_scores_weights= [{'aesthetic':0,'clip':-5,'hps':1},
#                      {'aesthetic':0,'clip':-2,'hps':1},
#                      {'aesthetic':0,'clip':-1,'hps':1},
#                      {'aesthetic':0,'clip':-0.5,'hps':1},
#                      {'aesthetic':0,'clip':0,'hps':1},
#                      {'aesthetic':0,'clip':1,'hps':1},
#                      {'aesthetic':0,'clip':3,'hps':1},
#                      {'aesthetic':0,'clip':5,'hps':1},
#                      {'aesthetic':0,'clip':100,'hps':1}]

images_list=[]
for i,scores_weights in enumerate(hps_scores_weights):
    prompt_with_keywords = ak4prompts_pipeline.keywords_ranking(prompt=prompt,scores_weights=scores_weights,topk=10)
    print(prompt_with_keywords)
    image = pipeline(
        prompt=prompt_with_keywords, num_inference_steps=4, guidance_scale=1.0,generator = generator
    ).images
    images_list += image

make_image_grid(images_list, rows=1, cols=9)